# *Colab Code Llama*
---

**Colab Code Llama**
An Assistant built on Code Llama.

**Colab Code Llama PRO**
- Upload chats and documents.
- Save chats.

## Setup and Installation

- You can run Colab Code Llama on a free Google Colab Notebook.
- Save a copy of this notebook: Go to File -> Save a copy in Drive. (optional, but needed if you want to make changes).
- Go to the menu -> Runtime -> Change Runtime Type - Select GPU (T4). Note that you'll have to limit context length to about 8k max for this. Or use an A100 40 GB for about 16k context length OR use an A100 80 GB for about 32k of context length. (for more compute, you can use Runpod [affiliate link here](https://tinyurl.com/yjxbdc9w)).
- Then go to Runtime -> Run all.
- It takes about 2 mins for the installation (which all happens in the cloud in this notebook).
- Once all cells have run, you'll find the chat interface at the bottom.

All of your data remains within your Google Drive and Google's computers.

In [ ]:
# Set the SYSTEM PROMPT
DEFAULT_SYSTEM_PROMPT = """You are a helpful pair-coding assistant."""
SYSTEM_PROMPT = DEFAULT_SYSTEM_PROMPT

print(SYSTEM_PROMPT)

You are a helpful pair-coding assistant.


In [ ]:
# Prompt user for context length
while True:
    user_input = input("Please enter the context length (must be between 256 and 128000): ")

    try:
        max_context = int(user_input)

        if 256 <= max_context <= 128000:
            break
        else:
            print("Context length must be between 256 and 128000. Please try again.")

    except ValueError:
        print("Invalid input. Please enter a number.")

# Your existing code for setting other variables
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

max_doc_length = int(0.75 * max_context)  # max doc length is 75% of the context length
max_doc_words = int(max_doc_length)

print(f"Using a context length of {max_context} and a document length of {max_doc_length}.")

Please enter the context length (must be between 256 and 128000): 8000
Using a context length of 8000 and a document length of 6000.


### Google Drive Mounting (optional)
- Allows you to download the model to Google Drive for faster startup next time.

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

## Install

In [ ]:
!pip3 install git+https://github.com/huggingface/transformers.git
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/
!pip3 install git+https://github.com/huggingface/optimum.git
!pip3 install git+https://github.com/HazyResearch/flash-attention.git#subdirectory=csrc/rotary
!pip3 install flash-attn --no-build-isolation

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-r3p5fpxy
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-r3p5fpxy
  Resolved https://github.com/huggingface/transformers.git to commit 21dc5859421cf0d7d82d374b10f533611745a8c5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.9 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.35.0.dev0-py3-none-any.whl size=7790405 sha256=abefb34d0f32351115cbeb3892b11bbffe0cdee77dca61f55ca1cbdaf3faacf3
  Stored in directory: /tmp/pip-ephem-whee

In [ ]:
!pip install -q -U pdfminer.six # could maybe add pre-built wheels to speed this up.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.3 MB/s eta 0:00:00


In [ ]:
!pip install transformers --upgrade

In [ ]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

### Import

In [ ]:
import transformers
import torch
import json
import os
from transformers import AutoTokenizer, TextStreamer

## Load Model
If you have connected to Google Drive, the model will load from there (unless this is your first time connecting, in which case the model will be saved to Drive).
- Takes about 2 mins first time around.
- Takes about 1 min the 2nd time onwards with Google Drive.

In [ ]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# model_id = "TheBloke/Yarn-Llama-2-7B-64K-GPTQ"
# model_id = "TheBloke/Yarn-Llama-2-13B-128K-GPTQ"
# model_id = "TheBloke/CodeLlama-13B-Instruct-GPTQ"
model_id = "./drive/MyDrive/codellama_13b_gptq"

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype=torch.float16,
                                             device_map="auto",
                                             revision="main",
                                             trust_remote_code=True,
                                             cache_dir='')

model = model.to_bettertransformer()

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

AttributeError: ignored

In [ ]:
print(model.config.max_position_embeddings)

4096


In [ ]:
print(model.config)

LlamaConfig {
  "_name_or_path": "TheBloke/Llama-2-7B-chat-GPTQ",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_length": 4096,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "batch_size": 1,
    "bits": 4,
    "block_name_to_quantize": null,
    "damp_percent": 0.01,
    "dataset": null,
    "desc_act": false,
    "disable_exllama": false,
    "group_size": 128,
    "model_seqlen": null,
    "module_name_preceding_first_block": null,
    "pad_token_id": null,
    "quant_method": "gptq",
    "sym": true,
    "tokenizer": null,
    "true_sequential": true,
    "use_cuda_fp16": false
  },
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "tie_word_embeddings": fa

# Token calculator

In [ ]:
def count_tokens(text):
    return len(tokenizer.encode(text))

# Model

## Set up the User Interface

In [ ]:
from IPython.display import display, HTML, clear_output, Markdown
import textwrap, json
import ipywidgets as widgets
import re, time
from google.colab import files
from pdfminer.high_level import extract_text
import io

In [ ]:
def generate_response(dialogs, temperature=0.01, top_p=0.9, logprobs=False):
    torch.cuda.empty_cache()
    # print(json.dumps(dialogs, indent=4))
    max_prompt_len = int(0.85 * max_context)
    max_gen_len = int(0.10 * max_prompt_len)

    prompt_tokens = []
    for dialog in dialogs:
        if dialog[0]["role"] != "system":
            dialog = [
                {
                    "role": "system",
                    "content": SYSTEM_PROMPT,
                }
            ] + dialog
        dialog_tokens = [tokenizer(
            f"{B_INST} {B_SYS}{(dialog[0]['content']).strip()}{E_SYS}{(dialog[1]['content']).strip()} {E_INST}",
            # f"{B_INST} {(dialog[1]['content']).strip()} {E_INST}", # Omits the system prompt altogether
            return_tensors="pt",
            add_special_tokens=True
        ).input_ids.to("cuda")]
        for i in range(2, len(dialog), 2):
            user_tokens = tokenizer(
                f"{B_INST} {(dialog[i+1]['content']).strip()} {E_INST}",
                return_tensors="pt",
                add_special_tokens=True
            ).input_ids.to("cuda")
            assistant_w_eos = dialog[i]['content'].strip() + tokenizer.eos_token
            assistant_tokens = tokenizer(
                            assistant_w_eos,
                            return_tensors="pt",
                            add_special_tokens=False
                        ).input_ids.to("cuda")
            tokens = torch.cat([assistant_tokens, user_tokens], dim=-1)
            dialog_tokens.append(tokens)
        prompt_tokens.append(torch.cat(dialog_tokens, dim=-1))

    input_ids = prompt_tokens[0]
    if len(input_ids[0]) > max_prompt_len:
        return "\n\n **The language model's input limit has been reached. Clear the chat and start afresh!**"

    # print(tokenizer.decode(input_ids[0], skip_special_tokens=True))

    with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
      generation_output = model.generate(
          input_ids=input_ids,
          do_sample=True,
          max_new_tokens=max_gen_len,
          temperature=temperature,
          top_p=top_p
      );

    new_tokens = generation_output[0][input_ids.shape[-1]:]
    new_assistant_response = tokenizer.decode(new_tokens, skip_special_tokens=True).strip();

    return new_assistant_response

In [ ]:
def print_wrapped(text):
    # Regular expression pattern to detect code blocks
    code_pattern = r'```(.+?)```'
    matches = list(re.finditer(code_pattern, text, re.DOTALL))

    if not matches:
        # If there are no code blocks, display the entire text as Markdown
        display(Markdown(text))
        return

    start = 0
    for match in matches:
        # Display the text before the code block as Markdown
        before_code = text[start:match.start()].strip()
        if before_code:
            display(Markdown(before_code))

        # Display the code block
        code = match.group(0).strip()  # Extract code block
        display(Markdown(code))  # Display code block

        start = match.end()

    # Display the text after the last code block as Markdown
    after_code = text[start:].strip()  # Text after the last code block
    if after_code:
        display(Markdown(after_code))

def grab_and_shorten_text(max_doc_length):

    uploaded = files.upload()

    file_name = list(uploaded.keys())[0]

    # Check the file extension
    if file_name.endswith('.txt'):
        text = uploaded[file_name].decode()
    elif file_name.endswith('.pdf'):
        pdf_bytes = io.BytesIO(uploaded[file_name])
        text = extract_text(pdf_bytes)
    else:
        raise ValueError('Unsupported file type. Please upload a .txt or .pdf file.')

    with alert_out:
        clear_output()  # Clear the previous alert
        print("Shortening the text...")

    tokens = tokenizer.encode(text, truncation=True, max_length=max_doc_length, return_tensors='pt');
    num_tokens_pdf = len(tokens)
    print(f"The PDF document has {num_tokens_pdf} tokens")

    num_tokens_system_prompt = count_tokens(SYSTEM_PROMPT)
    print(f"The system prompt has {num_tokens_system_prompt} tokens")

    # Count the number of total tokens
    total_num_tokens = num_tokens_pdf + num_tokens_system_prompt
    print(f"The total number of tokens is {total_num_tokens}")

    shortened_text = tokenizer.decode(tokens[0], skip_special_tokens=True)

    return file_name, shortened_text

dialog_history = [{"role": "system", "content": SYSTEM_PROMPT}]

button = widgets.Button(description="Send")
upload_button = widgets.Button(description="Upload .txt or .pdf")
text = widgets.Textarea(layout=widgets.Layout(width='800px'))

output_log = widgets.Output()

def on_button_clicked(b):
    user_input = text.value
    dialog_history.append({"role": "user", "content": user_input})

    text.value = ''

    # Change button description and color, and disable it
    button.description = 'Processing...'
    button.style.button_color = '#ff6e00'  # Use hex color codes for better color choices
    button.disabled = True  # Disable the button when processing

    with output_log:
        clear_output()
        for message in dialog_history:
            print_wrapped(f'**{message["role"].capitalize()}**: {message["content"]}\n')

    assistant_response = generate_response([dialog_history]);

    # Re-enable the button, reset description and color after processing
    button.description = 'Send'
    button.style.button_color = 'lightgray'
    button.disabled = False

    dialog_history.append({"role": "assistant", "content": assistant_response})

    with output_log:
        clear_output()
        for message in dialog_history:
            print_wrapped(f'**{message["role"].capitalize()}**: {message["content"]}\n')

button.on_click(on_button_clicked)

# Create an output widget for alerts
alert_out = widgets.Output()

# Define the 'Upload' button click event handler
def on_upload_button_clicked(b):

    file_name, uploaded_text = grab_and_shorten_text(max_doc_length)

    with alert_out:
        clear_output()  # Clear the previous alert
        print(f"Upload successful: {file_name}, processing the file...")

    user_input = f"Uploaded document [{file_name}]: {uploaded_text}"
    dialog_history.append({"role": "user", "content": user_input})

    time.sleep(0.1)  # slight delay to ensure order

    assistant_input = f"You have uploaded text from {file_name}"
    dialog_history.append({"role": "assistant", "content": assistant_input})

    with output_log:
        clear_output()
        for message in dialog_history:
            print_wrapped(f'**{message["role"].capitalize()}**: {message["content"]}\n')

    with alert_out:
        clear_output()  # Clear the previous alert
        # print(f"File processing completed.")

upload_button.on_click(on_upload_button_clicked)

clear_button = widgets.Button(description="Clear Chat")
text = widgets.Textarea(layout=widgets.Layout(width='800px'))

def on_clear_button_clicked(b):
    # Clear the dialog history
    dialog_history.clear()
    # Add back the initial system prompt
    dialog_history.append({"role": "system", "content": SYSTEM_PROMPT})
    # Clear the output log
    with output_log:
        clear_output()

clear_button.on_click(on_clear_button_clicked)

def save_chat(b):
    # Serialize the chat history into a JSON string
    chat_json = json.dumps(dialog_history)

    # Write the chat history to a temporary file
    with open('chat_history.json', 'w') as f:
        f.write(chat_json)

    # Download the file
    files.download('chat_history.json')

save_button = widgets.Button(description="Save Chat")
save_button.on_click(save_chat)

In [ ]:
# Define the function to upload chat
def upload_chat(b):
    # Upload the file
    uploaded = files.upload()

    # Get the file name
    file_name = list(uploaded.keys())[0]

    # Ensure the file is a .json file
    if not file_name.endswith('.json'):
        print('Error: Incorrect file type. Please upload a .json file.')
        return

    # Load the content of the file
    chat_data = uploaded[file_name].decode()

    # Load the JSON data from the file
    try:
        global dialog_history
        dialog_history = json.loads(chat_data)
    except json.JSONDecodeError:
        print('Error: File is not in the correct format. Please upload a properly formatted .json file.')
        return

    with output_log:
        clear_output()
        for message in dialog_history:
            print_wrapped(f'**{message["role"].capitalize()}**: {message["content"]}\n')

# Create the upload button and set the on_click event handler
upload_chat_button = widgets.Button(description="Upload Chat")
upload_chat_button.on_click(upload_chat)


In [ ]:
from IPython.display import display, HTML
from ipywidgets import HBox, VBox

# Create the title with HTML
title = f"<h1 style='color: #ff6e00;'>Colab Code Llama 🦙 PRO</h1> <p>(Max context of: {max_context}. Uploaded files will be shortened to {max_doc_words} tokens)</p>"

# Assuming that output_log, alert_out, and text are other widgets or display elements...
first_row = HBox([button, clear_button, upload_button])  # Arrange these buttons horizontally
second_row = HBox([save_button, upload_chat_button])  # Arrange these buttons horizontally

# Arrange the two rows of buttons and other display elements vertically
layout = VBox([output_log, alert_out, text, first_row, second_row])

# Chat with Colab Code Llama

In [ ]:
display(HTML(title))  # Use HTML function to display the title
display(layout)

Saving berkshire23.txt to berkshire23.txt
